In [3]:
%matplotlib widget
#%matplotlib inline


import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/Users/elka127/Documents/code/rh")
#set_matplotlib_formats('svg')
# plt.rc('legend', frameon=False)
# plt.rc('figure', figsize=(7, 7 / 1.75)) # Larger figure sizes
# plt.rc('font', size=12)

# With the Bifrost Snapshot or with existing atmosperes' models

```
# There are four 1D models of flares on M-dwarfs (/mn/stornext/u3/matsc/rh/Atmos/mflares/). 
# These are:

# radyn_out.F13_dpl_GRID_pt1s.ncdf
# tx.m2F12-37.max2F12_Ec37_d3.ncdf
# tx.m5F12-37.max5F12_Ec37_d3.ncdf
# tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf

# the three "tx" models all have a low cut-off energy of 37 keV and a power index of 3 
# (see Allred et al. 2015) 
# but vary in maximum beam energy: 2e12, 5e12 and 1e13 erg/cm^2/s. 

#The model from Kowalski et al. 2015 is radyn_out.F13_dpl_GRID_pt1s.ncdf.
```



In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/code/rh/Atmos/"

In [ ]:
# to read xarray.Dataset and see dimentions, coordinates, data variables and attributes 
import xarray
atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
atmos
# out = xarray.open_dataset("/Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5")
# out
#output_indata.hdf5 file.

#output_ray.hdf5 file.

```
#netcdf4 to hdf5
```

In [ ]:
atmos.to_netcdf("newfile.hdf5", format='NETCDF4')

```
# виджет хелиты все равно не работает даже с переделанным файлом из netcdf
# строит только 1 график
```

In [ ]:
from helita.vis import rh15d_vis

# hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# H = hf['z'][0,:]
#print(H)
rh15d_vis.InputAtmosphere("/Users/elka127/Documents/uio/repository/jupyter/newfile.hdf5")



# Plotting atmospheres in nz

In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/code/rh/Atmos"

In [ ]:
#hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
hf = h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf,"r")
height = hf['z'][0,:,0,:] # geometric height
time = hf['x'][:]
#print(height.shape())
temp = hf['temperature']
vlos = hf['velocity_z'] # LOS_V
# eden = hf['electron_density']

# snapshot_number = hf["snapshot_number"]
# nhydr = hf["nhydr"]
# x = hf["x"]
# z = hf["z"]
print(height)


In [ ]:
# to plot 2-d atmospheres with height scale as index 
fig, ax = plt.subplots()
#atmos.temperature.plot.hist()
atmos.temperature.plot()
#atmos.velocity_z.plot()
#atmos.electron_density.plot()
#im=axs[0].imshow(temp[0,:,0,:])

#ax.plot(height / 1e6, time)  # height in Mm
ax.set_xlabel('H (Mm)')
ax.set_ylabel('time')
#ax.set_yscale('log')
#ax.set_xscale('log')

#ax.set_xlabel('height')
#ax.set_ylabel('time')


```
# additional way to see variables
```

In [ ]:
hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

In [ ]:
# To see what data is in this file, we can call the keys() method on the file object.
hf.keys() # variables in the Bifrost snapshot

# Plotting atmospheres in real height

In [ ]:
#from helita.sim import rh15d_vis
hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
H = hf['z'][0,:]
#print(H)
rh15d_vis.InputAtmosphere2D("/Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5")
#nz = H
#atmos.temperature.plot.hist()
#fig, ax = plt.subplots()
#atmos.temperature.plot()
#atmos.velocity_z.plot()

# atmos.electron_density.plot()
# ax.set_xlabel('height')
# ax.set_ylabel('time')
#hf = h5py.File(rhRepoPath / "rh15d/run/output/output_indata.hdf5","r")
#hf.keys()
# atmos = hf['atmos']
# height = hf['height']
# inputfile = hf['input']
# mpi = hf['mpi']

# print(atmos)
# print("///")
# print(height)
# print("///")
# print(inputfile)
# print("///")
# print(mpi)

#Blos = hf['B_z'] # LOS B
#H = hf['z'][0,:] # geometric height
#snapshot_number = hf["snapshot_number"]
#nhydr = hf["nhydr"]

#import xarray
#inputs = xarray.open_dataset(rhRepoPath / "rh15d/run/output/output_indata.hdf5")


In [694]:
%matplotlib widget


hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m5F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig, ax=plt.subplots()

# ax1.plot(z1t[:,0],np.log10(tg1t[:,100]),label='10.0 sec')
# ax1.plot(z1t[:,0],np.log10(tg1t[:,75]),label='7.5 sec')
# ax1.plot(z1t[:,0],np.log10(tg1t[:,60]),label='6.0 sec')

ax.plot(z1t[:,0],np.log10(tg1t[:,40]),'k--', label='4.0 sec')
ax.plot(z1t[:,0],np.log10(tg1t[:,22]),'k-',label='2.2 sec')
ax.plot(z1t[:,0],np.log10(tg1t[:,12]),'r-.',label='1.2 sec')
ax.plot(z1t[:,0],np.log10(tg1t[:,2]),'b-',label='0.2 sec')
ax.plot(z1t[:,0],np.log10(tg1t[:,0]),'g-',label='0 sec')

# ax1.plot(z1t[:,0],vz1t[:,135],label='13.5 sec')
# ax1.plot(z1t[:,0],vz1t[:,90],label='9.0 sec')
# ax1.plot(z1t[:,0],vz1t[:,60],label='6.0 sec')

# ax1.plot(z1t[:,0],vz1t[:,40],label='4.0 sec')
# ax1.plot(z1t[:,0],vz1t[:,22],label='2.2 sec')
# ax1.plot(z1t[:,0],vz1t[:,12],label='1.2 sec')
# ax1.plot(z1t[:,0],vz1t[:,2],label='0.2 sec')
# ax1.plot(z1t[:,0],vz1t[:,0],label='0 sec')

# ax1.plot(z1t[:,0],np.log10(ed1t[:,130]),label='13.0 sec')
# ax1.plot(z1t[:,0],np.log10(ed1t[:,100]),label='10.0 sec')
# ax1.plot(z1t[:,0],np.log10(ed1t[:,75]),label='7.5 sec')

# ax1.plot(z1t[:,0],np.log10(ed1t[:,40]),label='4.0 sec')
# ax1.plot(z1t[:,0],np.log10(ed1t[:,22]),label='2.2 sec')
# ax1.plot(z1t[:,0],np.log10(ed1t[:,12]),label='1.2 sec')
# ax1.plot(z1t[:,0],np.log10(ed1t[:,2]),label='0.2 sec')
# ax1.plot(z1t[:,0],np.log10(ed1t[:,0]),label='0 sec')




plt.xlabel('Height (Mm)')

plt.ylabel('lg(Temperature (K))')
#plt.ylabel('Velocity (m/s)')
#plt.ylabel('lg(Electron Density ())')
#plt.title('m2F12-37 Atmos’)
ax.legend()  # Add a legend.

plt.savefig("temperature_.svg", format="svg")
#plt.savefig("velocity_z_.svg", format="svg")
#plt.savefig("electron_density_.svg", format="svg")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Atoms for CaII

```
# Nmetal
   10

# Metals
#  model file              ACTIVE/PASSIVE  INITIAL_SOLUTION   population file


# ../../Atoms/CaII.atom            PASSIVE    ZERO_RADIATION
  ../../Atoms/H_6.atom             PASSIVE    ZERO_RADIATION
  ../../Atoms/MgII-IRIS.atom       PASSIVE    ZERO_RADIATION
  ../../Atoms/CaII_PRD.atom        ACTIVE     ZERO_RADIATION
  ../../Atoms/Si.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Al.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Fe.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/He.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/N.atom               PASSIVE    LTE_POPULATIONS
  ../../Atoms/Na.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/S.atom               PASSIVE    LTE_POPULATIONS
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

wave.sel(wavelength=36, method="nearest")
index500 = np.argmin(np.abs(wave.data - 360))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

# Writing ray.input file

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

#wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

```
#if we want to save particular wavelenghts, 500 for example, and get its index
```

In [ ]:
wave.sel(wavelength=500, method='nearest')

In [ ]:
index500 = np.argmin(np.abs(wave.data - 500))

```
# To save this into a file ray.input we do:
```

In [ ]:
f = open('ray.input', 'w')  
# this will overwrite any existing file!
f.write('1.00\n')
output = str(len(indices) + 1)
for ind in indices:
    output += ' %i' % ind
output += ' %i\n' % index500 
f.write(output)
f.close()

# Create own wavetable file

```
# Writes RH wave file (in xdr format). All wavelengths should be in nm.

# Parameters
# ----------
# start: number
#     Starting wavelength.
# end: number
#     Ending wavelength (non-inclusive)
# step: number
#     Wavelength separation
# outfile: string
#     Name of file to write.
# ewave: 1-D array, optional
#    Array of existing wavelengths. Program will make discard points
#    to make sure no step is enforced using these points too.
# air: boolean, optional
#     If true, will at the end convert the wavelengths into vacuum
#     wavelengths.
```

In [ ]:
def make_wave_file(my_wave, start=20, end=4000, step=,5 new_wave=None,
                   ewave=None, air=False):
                   rh15d.make_wave_file('my_wave.wave', ewave=my_waves)

In [ ]:
#from helita.sim import rh15d
#read exisitng file if needed
MY_wave = rh15d.read_wave_file(rhRepoPath / "Atoms/wave_files/My_wave.wave")


In [ ]:
#import xarray
#atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
print(MY_wave)

In [ ]:
MY_wave.shape

# Selecting wavelengths for detailed output

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]
#If we want to see also one particular wavelength (500 in example)
wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

# Intencity plot for Ca II

In [4]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray 

In [678]:
%matplotlib widget
#%matplotlib inline
data.ray.intensity.plot()
#plt.legend(loc="upper left")
plt.xlabel("Wavelenght nm")
plt.ylabel("Time, s*10")
#plt.axis([20, 1000, 0, 62])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Time, s*10')

In [5]:
%matplotlib widget
#%matplotlib inline
data.ray.intensity.plot()
plt.savefig("test.svg", format="svg")
plt.axis([392.8, 394.0, 0, 2e-9])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(392.8, 394.0, 0.0, 2e-09)

In [6]:
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()
# tau1.plot()

NameError: name 'h5py' is not defined

In [7]:
%matplotlib widget
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394)]
#If we want to see also one particular wavelength (500 in example)
# wave.sel(wavelength=500, method="nearest")
# index500 = np.argmin(np.abs(wave.data - 500))
# tau = hf['tau_one_height']
# z1t = hf['height'][0,:,0,:]/1.e6 
# intens = hf['intensity']
# fig1, ax1=plt.subplots()

# ax1.plot(z1t[:,0],np.log10(tg1t[:,100]),label='10.0 sec')
# ax1.plot(z1t[:,0],np.log10(tg1t[:,75]),label='7.5 sec')
# ax1.plot(z1t[:,0],np.log10(tg1t[:,60]),label='6.0 sec')
# print(tau)
# ax1.plot(tau[:,0],intens[:,0])


--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray 

In [8]:
%matplotlib widget

fig, ax=plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6, 'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6, 'k-',label='0.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6, 'r-',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6, 'b-',label='2.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6, 'g-',label='4. sec')

ax.legend() 
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')
plt.title(r'$\tau=1$')
plt.savefig("Tau_one_height_.svg", format="svg")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

```
#To make a plot for intensity at (x, y) = (?, 0), x - time column
```

In [20]:
%matplotlib widget

fig, ax=plt.subplots()
#plt.figure()
# for i in range(0,2,12,22,38):
#     plt.plot(
#         data.ray.intensity[i, 0],
#         label='Time %i' % (i)
#     )
# data.ray.intensity[100, 0].plot(label='10 sec')
# data.ray.intensity[75, 0].plot(label='7.5 sec')
# data.ray.intensity[60, 0].plot(label='6 sec')

ax.plot(wave[indices],data.ray.intensity[0,0,indices],'k--',label='0 sec')
ax.plot(wave[indices],data.ray.intensity[2,0,indices],'k-',label='0.2 sec')
ax.plot(wave[indices],data.ray.intensity[12,0,indices],'r-',label='1.2 sec')
ax.plot(wave[indices],data.ray.intensity[22,0,indices],'b-',label='2.2 sec')
ax.plot(wave[indices],data.ray.intensity[40,0,indices],'g-',label='4 sec')


# data.ray.intensity[40, 0].plot(label='4 sec')
# data.ray.intensity[22, 0].plot(label='2.2 sec')
# data.ray.intensity[12, 0].plot(label='1.2 sec')
# data.ray.intensity[2, 0].plot(label='0.2 sec')
# data.ray.intensity[0, 0].plot(label='0 sec')


ax.legend()  # Add a legend.

plt.savefig("intensity_.svg", format="svg")
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Intensity (Stokes I), W/(m2 Hz sr) (Mm)')
#ax.set_yscale('log')
plt.axis([392.8, 394, 0, 8e-7])
plt.savefig("Intencity_.svg", format="svg")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

```
#Flux from Intencity
```

In [561]:
# from scipy import integrate
# from integrate import quad, dblquad
from scipy.integrate import quad, dblquad
fluxes = []

# def u(x):
#     #print(i, ii)
#     return np.cos(x)*data.ray.intensity[i,0,ii].data
#print(u(0))
#x = np.cos(y)
#print(data.ray.intensity)
#print(data.ray.intensity.shape, data.ray.intensity.count().data)
for i in range(data.ray.intensity.x.count().data):
    for ii in range(data.ray.intensity.wavelength.count().data):
        #print(data.ray.intensity[i,0,ii].data)
        flx, err = dblquad(lambda x, y: np.sin(x)*np.cos(x)*data.ray.intensity[i,0,ii].data, 0, np.pi, lambda x: 0, lambda x: np.pi/2)
        #integrate.quad(lambda x: x*data.ray.intensity[i,0,ii].data, -1, 1) #weight="cos", wvar=1)
        #fluxes.append(round(flx,5))
        #print(flx)
        fluxes.append(flx)
#print(fluxes)
np.save("Flux.npy",fluxes)

In [703]:
from scipy.integrate import quad, dblquad
fluxes = []

for i in [0,2,12,22,40]:
    for ii in range(data.ray.intensity.wavelength.count().data):
        #print(data.ray.intensity[i,0,ii].data)
        flx, err = dblquad(lambda x, y: np.sin(x)*np.cos(x)*data.ray.intensity[i,0,ii].data, 0, np.pi, lambda x: 0, lambda x: np.pi/2)
        #integrate.quad(lambda x: x*data.ray.intensity[i,0,ii].data, -1, 1) #weight="cos", wvar=1)
        #fluxes.append(round(flx,5))
        #print(flx)
        fluxes.append(flx)
#print(fluxes)
np.save("Flux.npy",fluxes)

In [704]:
fluxesNP = np.array(fluxes)
fluxes2d = fluxesNP.reshape(5,1220)
print(fluxes2d.shape)
#np.save("fluxes2d", fluxes2d)

(5, 1220)


In [708]:
%matplotlib widget
fig, ax=plt.subplots()
#np.load("Flux.npy")

# fluxes2d[40, 0].plot(label='4 sec')
# fluxes2d[22, 0].plot(label='2.2 sec')
# fluxes2d[12, 0].plot(label='1.2 sec')
# fluxes2d[2, 0].plot(label='0.2 sec')
# fluxes2d[0, 0].plot(label='0 sec')
ax.plot(wave[indices],(fluxes2d[0,:]),'k--',label='0 sec')
ax.plot(wave[indices],(fluxes2d[1,:]),'k-',label='0.2 sec')
ax.plot(wave[indices],(fluxes2d[2,:]),'r-',label='1.2 sec')
ax.plot(wave[indices],(fluxes2d[3,:]),'b-',label='2.2 sec')
ax.plot(wave[indices],(fluxes2d[4,:]),'g-',label='4.0 sec')


ax.legend()
plt.xlabel("Wavelenght nm")
plt.ylabel("Flux, W/(m2 Hz)")
plt.axis([0, 4000, 0, 1.5e-6])

plt.savefig("flux_.svg", format="svg")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [617]:
#print(data.ray.intensity.wavelength[1219].data)
fluxesSum, err = integrate.quad(
    lambda x: fluxes2d[0,x],
    0,#data.ray.intensity.wavelength[0].data,
    1219#data.ray.intensity.wavelength[1219].data
)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

```
#To make a plot for intensity at a fixed wavelength
```


In [19]:
%matplotlib widget
data.ray.intensity.sel(wavelength=360, method='nearest').plot()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
rh15d_vis.InputAtmosphere(data)

# Calculating optical depths

In [11]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
print(data.ray.wavelength_selected)
print("-")
print(data.ray.wavelength_selected.data)

# first column
height = data.atmos.height_scale[0, 0].dropna('height')  
# index of 500 nm
index360 = np.argmin(np.abs(data.ray.wavelength_selected.data - 360))  

tau360 = cumtrapz(data.ray.chi[0, 0, :, index360].dropna('height'), x=-height)
# ensure tau500 and height have same size
tau360 = np.concatenate([[1e-20], tau360])  

 

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
<xarray.DataArray 'wavelength_selected' (wavelength_selected: 43)>
array([392.926626, 393.079987, 393.190157, 393.26934 , 393.326292, 393.367295,
       393.396854, 393.418205, 393.433666, 393.444901, 393.453105, 393.459133,
       393.463602, 393.466951, 393.469496, 393.471466, 393.473022, 393.474282,
       393.475329, 393.476223, 393.477007, 393.477713, 393.478419, 393.479204,
       393.480098, 393.481145, 393.482405, 393.483961, 393.48593 , 393.488476,
       393.491825, 393.496293, 393.502322, 393.510526, 393.521761, 393.537222,
       393.558572, 393.588132, 393.629135, 393.686087, 393.76527 , 393.87544 ,
       360.102708])
Coordinates:
  * wavelength_selected  (wavelength_selected) float64 392.9 393.1 ... 360.1
Attributes:
    units:  

In [12]:
%matplotlib widget

fig, ax = plt.subplots()
ax.plot(height / 1e6, tau360)  # height in Mm

plt.ylabel(r'$\tau$$_{360}$')
plt.xlabel('H (Mm)')

ax.set_yscale('log')
ax.axhline(y=1,linestyle='dashed',color='black')
#ax.axvline(x=0.5,linestyle='dashed',color='black')
plt.savefig("tau360_.svg", format="svg")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot the departure coefficients for the levels of Ca II



In [15]:
%matplotlib widget

plt.figure()
for i in range(5):
    plt.plot(data.atmos.height_scale[22,0]/1e6, 
             data.atom_CA.populations[i, 22, 0]/data.atom_CA.populations_LTE[i, 22, 0],
             label='Level %i' % (i + 1))
plt.legend(loc="upper left")
plt.xlabel("Departure coefficients at 2.2 sec")
plt.ylabel("H (Mm)")

# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
# fig, ax = plt.subplots()
# ax.plot(tau500,data.atom_CA.populations[1, 0, 0, 0]/data.atom_CA.populations_LTE[1, 0, 0, 0], label='Level %1' % (1)) 
# plt.legend(loc="upper left")
# plt.xlabel("tau500")
# plt.ylim(0.05, 2)
# plt.ylabel("Departure coefficients")
# plt.ylim(0.05, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'H (Mm)')

In [16]:
%matplotlib widget

fig, ax = plt.subplots()
ax.plot(data.atmos.height_scale[0, 0].dropna('height')/1e6, data.atom_CA.populations[0,0,0,:].dropna('height')/data.atom_CA.populations_LTE[0,0,0,:].dropna('height'),'r--',label=r'b$_{l}$')
# ax.plot(data.atmos.height_scale[0, 0].dropna('height')/1e6, np.log10(data.atom_CA.populations[0,3,0,:].dropna('height')/data.atom_CA.populations_LTE[0,3,0,:].dropna('height')),'k-',label=r'b$_{u}$')
plt.legend(loc='best')
plt.xlabel('H (Mm)')
plt.ylabel(r'Log (b$_{i}$)')
#plt.xlim([0,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Log (b$_{i}$)')

# tau=1

In [17]:
%matplotlib widget

fig, ax = plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6,'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6,'k-',label='0.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6,'r-',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6,'b-',label='2.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6,'g-',label='4 sec')

plt.legend(loc='best')

plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Height (Mm)')

In [479]:
height = data.ray.height
fig, ax = plt.subplots()
ax.plot(height/)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# от тау 500

In [ ]:
# xVals = data.atmos.height_scale[0,0]/1e6
# with open("/tmp/x.txt", "w") as f:
#     for x in xVals.data:
#         f.write(f"{x}\n")

print(data.atom_CA.populations[0,22,0])
# yVals = data.atom_CA.populations[0,0,0]
# with open("/tmp/y.txt", "w") as f:
#     for y in yVals.data:
#         f.write(f"{y}\n")


#print(data.atom_CA.populations[0,0,0][247])
# print("- - - - -")
# print(data.atom_CA.populations[1,0,0])
# print("- - - - -")
# print(data.atom_CA.populations[2,0,0])

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(
    data.atmos.height_scale[0,0]/1e6,
    data.atom_CA.populations[0,0,0]
)

def animate(i):
    #line.set_xdata(x)
    line.set_ydata(data.atom_CA.populations[0, i, 0][0])
    return line,

ani = animation.FuncAnimation(
    fig,
    animate,
    np.arange(52),
    #blit=True,
    repeat=False
)
plt.show()

In [ ]:
plt.figure()
for i in range(5):
    plt.plot(
        data.atmos.height_scale[0,0]/1e6, 
        data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
        label='Level %i' % (i + 1)
    )
plt.legend(loc="upper left")
plt.ylabel("Height (Mm)")
plt.xlabel("Departure coefficients")
plt.xlim(0,11)
plt.ylim(-20,10000)
#/data.atom_CA.populations_LTE[i, 100, 0]

# Source function

In [21]:
rh15d_vis.SourceFunction(data);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='wavelength', max=42), Checkbox(value=True, description='…

# Попытка сделать анимацию на базе кода матса

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d
import h5py

hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig1, ax1=plt.subplots()
#ax1.plot(vz1t[:,22],np.log10(tg1t[:,22]))
ax1.plot(vz1t[:,0],(tg1t[:,0]))

plt.xlabel('Height (Mm)')
plt.ylabel('(Velocity (m/s))')

#plt.ylabel('lg(Temperature (K))')
#plt.title('Figure 1’)

# further nonsence with atmo


In [ ]:
rh15d_vis.InputAtmosphere("/Users/elka127/Documents/uio/ast5210/docs/untitled folder/rh_ast5210/Atmos/bifrost_cb24bih_s385_cut.hdf5");
#rh15d_vis.InputAtmosphere("/Users/elka127/Documents/uio/ast5210/docs/untitled folder/rh_ast5210/Atmos/bifrost_cb24bih_s385_cut.hdf5");
# import h5py
#   data = h5py.File(rhRepoPath / "rh15d/run/output/output_indata.hdf5","r")
#   for group in data.keys():
#        print(group)

In [ ]:
atmosTemp = atmos.temperature 
#atmostemp

# print(atmosTemp[0,:,:,248])
# print()
# print(atmosTemp[0,:,:,247])

# 1 - nt(1) - ?
# 2 - nx(52) - время
# 3 - ny(1) - ?
# 4 - nz(249) - высота

# sm1 = atmosTemp[0,0,0,248]
# print(sm1)
# sm2 = atmosTemp[0,0,0]
# print(sm2)

#print(atmosTemp[0,:,:,:])

# ---

atmosHeight = atmos.z
atmosHeight

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,5),facecolor='w', edgecolor='k')
#fig.subplots_adjust(hspace = 0.1,wspace=0.25,left=0.05,right=0.95,top=0.93,bottom=0.07)
#axs.ravel()

# im=axs[0].imshow(atmosTemp[0,:,:,248],cmap='plasma')
# im=axs[1].imshow(atmosTemp[0,:,:,247],cmap='plasma')
# im=axs[2].imshow(atmosTemp[0,:,:,246],cmap='plasma')

im=axs[0].plot(atmosTemp[0,:,:,247])
im=axs[1].plot(atmosHeight[0,:,:,247])
#im=axs[2].plot(atmosTemp[0,:,:,246])

# divider0 = make_axes_locatable(axs[0])
# cax0 = divider0.append_axes("right", size="5%", pad=0.05)
# cbar0=plt.colorbar(im,cax=cax0)
# cbar0.set_label('T (K)')

# divider1 = make_axes_locatable(axs[1])
# cax1 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar1=plt.colorbar(im,cax=cax1)
# cbar1.set_label('T (K)')

#axs[0].set_xlabel('pixels')
axs[0].set_ylabel('pixels')

In [ ]:
import h5py
data = h5py.File(rhRepoPath / "rh15d/run/output/output_indata.hdf5","r")
for group in data.keys():
    print(group)

In [ ]:
import xarray
atmos = xarray.open_dataset("/Users/elka127/Documents/uio/ast5210/docs/untitled folder/rh_ast5210/Atmos/bifrost_cb24bih_s385_cut.hdf5")
atmos

In [ ]:
atmos.temperature.plot()

In [ ]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/uio/project-Mflare/mflares/"

from helita.sim import bifrost
data = bifrost.BifrostData("radyn_out.F13_dpl_GRID_pt1s", snap=300, fdir=dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")

In [ ]:
#H = hf['z'][0,:] 
# geometric height
plt.figure()
plt.plot(
        atmos.height_scale[0,0]/1e6, 
        atmos.electron_density[0, 0, 0]
)
print(atmos.height_scale[0,0]/1e6)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# height = hf['z'][0,:,0,:] # geometric height
# time = hf['x'][:]
# temp = hf['temperature']
# vlos = hf['velocity_z'] # LOS_V
# eden = hf['electron_density']

hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig, axs =plt.subplots(1,3,figsize=(15,5),facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.25,wspace=0.25,left=0.05,right=0.95,top=0.93,bottom=0.07)
axs.ravel()
im=axs[0].matshow(tg1t)
divider1 = make_axes_locatable(axs[0])
cax0 = divider1.append_axes("right", size="5%", pad=0.05)
cbar=plt.colorbar(im,cax=cax0)
cbar.set_label('T (K)')
axs[0].set_xlabel('time')
axs[0].set_ylabel('height')

# im=axs[1].matshow(vlos[0,:,0,:]/1e3,cmap='bwr',vmax=1.5,vmin=-1.5)
# divider1 = make_axes_locatable(axs[1])
# cax0 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar=plt.colorbar(im,cax=cax0)
# cbar.set_label('V (km/s)')
# axs[1].set_xlabel('height')
# axs[1].set_ylabel('time')

# im=axs[2].matshow(eden[0,:,0,:]*1e4,cmap='BrBG',vmax=20,vmin=-100)
# divider1 = make_axes_locatable(axs[2])
# cax0 = divider1.append_axes("right", size="5%", pad=0.05)
# cbar=plt.colorbar(im,cax=cax0)
# cbar.set_label('electron density (G)')
# axs[2].set_xlabel('height')
# axs[2].set_ylabel('time')